# Plugins

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.45.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.45.0"

##### Plugins can be used to expose functionality to your model. This works well, when you already have an application built out, and would like to add AI to the app.

In [3]:
using System.ComponentModel;
using Microsoft.SemanticKernel;

public class MathFunctions
{
    [KernelFunction, Description("Calculates the square of a number.")]
    public static double square ([Description("The number to calculate square of")] double x)
    {
        return x * x;
    }

    [KernelFunction, Description("Calculates the cube of a number.")]
    public static double cube ([Description("The number to calculate cube of")] double x)
    {
        return x * x * x;
    }
}

 public class TimeInformation
{
    [KernelFunction]
    [Description("Retrieves the current time in UTC.")]
    public string GetCurrentUtcTime() => DateTime.UtcNow.ToString("R");
}

Let us have LLM utilize the functions

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;


#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
ILoggerFactory myLoggerFactory = NullLoggerFactory.Instance;
var builder = Kernel.CreateBuilder();
builder.Services.AddSingleton(myLoggerFactory); //Add logging!
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();
#pragma warning disable SKEXP0001 
// Lets the model choose the function to call based on the prompt
OpenAIPromptExecutionSettings settings = new() { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };

// We import the MathFunctions class to the kernel
// This will allow us to use the functions defined in the class
kernel.Plugins.AddFromType<MathFunctions>();
kernel.Plugins.AddFromType<TimeInformation>();

Console.WriteLine ( await kernel.InvokePromptAsync("What is the square of 25", new (settings)) );
Console.WriteLine ( await kernel.InvokePromptAsync("How many days until Christmas. Explain your thinking", new (settings)) );


#### What if you have a service you need to use to supply functionality? You can use OpenAPI!

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;


#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
ILoggerFactory myLoggerFactory = NullLoggerFactory.Instance;
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(model, apiKey, orgId);
var kernel = builder.Build();

var stream = File.OpenRead("resources/api/repairs.json");
var plugin = await kernel.ImportPluginFromOpenApiAsync("RepairService", stream!);

// This will allow us to use the functions defined in the class
Console.WriteLine ( await kernel.InvokePromptAsync("Can you get me a list of all repairs?", new (settings)) );